<a href="https://www.kaggle.com/code/akouaorsot/full-stack-data-science-ba-flight-reviews?scriptVersionId=140880860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

### Project 1: British Airways Passengers' Reviews between 2016-2023

With data cleaning taking up a considerable chunk of data science workflow, it is critical to know how to assess, improve, and produce data quality. Every subsequent steps from analysis to machine learning modeling hinges on having good data. And so, this project will focus on practicing best practices when it comes to data cleaning to ensure higher quality and usability. 

In that vein, we examined various sources, especially the following three in an attempt to draft a comprehensive map for the cleaning process: 
* [World Bank Data Cleaning Checklist](https://dimewiki.worldbank.org/Checklist:_Data_Cleaning)
* [Mariann Beargie on Medium](https://mariannbea.medium.com/are-you-sure-your-data-is-completely-clean-use-this-checklist-to-help-8cb415a8b0e0)
* [DataCamp Data Cleaning Infographic](https://www.datacamp.com/blog/infographic-data-cleaning-checklist)

Based on the sources above, we modelled our steps and ensure that we could produce a score on the scale of 1 to 10 before and after cleaning.

# Loading

In [1]:
## Importing Libraries
import os           # operating system
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import category_encoders as ce # Encode Categorical Variables

import matplotlib.pyplot as plt # data visualization

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
## Importing the dataset
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))

paths

['/kaggle/input/british-airways-passenger-reviews-2016-2023/British_Airway_Review.csv']

# Overview

In [3]:
# Snapshot of the first 5 rows
df = pd.read_csv(paths[0])
df.head()

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller
0,✅ Trip Verified | I had the most fantastic BA...,1st August 2023,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure
1,✅ Trip Verified | Couldn’t book in online. Ar...,31st July 2023,United Kingdom,Economy Class,no,3,Rome to Heathrow,Solo Leisure
2,✅ Trip Verified | London Heathrow to Mumbai in...,31st July 2023,Iceland,Business Class,yes,3,Gatwick to Venice,Solo Leisure
3,"✅ Trip Verified | Keflavík, Iceland to London ...",31st July 2023,Iceland,Business Class,yes,5,London to Luanda,Couple Leisure
4,✅ Trip Verified | Terrible Experience with Bri...,29th July 2023,Canada,Economy Class,no,5,Denver to Heathrow,Family Leisure


In [4]:
# Shape of data: No. of rows and columns
no_rows, no_cols = df.shape

# Check for missing values
null_values = df.isnull().sum().sum()

# No. of duplicates values
duplicates_n = df[df.duplicated()].shape[0]


# Diagnostics results
data_check = """
- The data has {0} rows and {1} columns
- There are {2} null/na values and {3} duplicated rows. 
""" 

print(data_check.format(no_rows, no_cols,
                        null_values, duplicates_n))


- The data has 2500 rows and 8 columns
- There are 0 null/na values and 0 duplicated rows. 



In [5]:
# # Categorical Variables
# """
# - col:
#     - 
# """
# Unique value counts for categorical columns
for col in df.columns[2:]:
    print(col)
    col_cats_count = df[col].value_counts(normalize=True, sort=True)
    n_cat = len(col_cats_count.keys())
    freq = dict(col_cats_count)
    if n_cat <= 10:
        print(n_cat)
        print(freq)
    else:
        print(n_cat)

country
67
seat_type
4
{'Economy Class': 0.5452, 'Business Class': 0.3016, 'Premium Economy': 0.0992, 'First Class': 0.054}
recommended
2
{'no': 0.648, 'yes': 0.352}
stars
5
{3: 0.4548, 5: 0.2728, 1: 0.0908, 9: 0.0908, 7: 0.0908}
route
10
{'Heathrow to Las Vegas': 0.1, 'Rome to Heathrow': 0.1, 'Gatwick to Venice': 0.1, 'London to Luanda': 0.1, 'Denver to Heathrow': 0.1, 'BKK to LHR': 0.1, 'London to Tampa': 0.1, 'London to Sydney': 0.1, 'LHR to CPT': 0.1, 'BLR to LHR': 0.1}
type_of_traveller
4
{'Couple Leisure': 0.4, 'Family Leisure': 0.2, 'Solo Leisure': 0.2, 'Business': 0.2}


# Exploratory Data Analysis

# Statistical Analysis

# Feature Transformation/Engineering

In [6]:
# Create a different column "trip verified" to seperate them properly
df['trip_verified'] = np.where(df['reviews'].str.contains("Trip Verified"), 1, 0) 

# Capture the main content of the reviews and create a list of each word 
df['reviews_verif_drop'] = df['reviews'].str.replace("✅ Trip Verified \| ","").str.strip()
df['reviews_bag_of_words'] = df['reviews_verif_drop'].str.split(" ")

# Converting 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# New snapshot of first 5 rows after transformation
df.head()

/tmp/ipykernel_20/2525839514.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews_verif_drop'] = df['reviews'].str.replace("✅ Trip Verified \| ","").str.strip()


,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller,trip_verified,reviews_verif_drop,reviews_bag_of_words
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure,1,I had the most fantastic BA Flight today. The ...,"[I, had, the, most, fantastic, BA, Flight, tod..."
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,Economy Class,no,3,Rome to Heathrow,Solo Leisure,1,Couldn’t book in online. Arrived at check in t...,"[Couldn’t, book, in, online., Arrived, at, che..."
2,✅ Trip Verified | London Heathrow to Mumbai in...,2023-07-31,Iceland,Business Class,yes,3,Gatwick to Venice,Solo Leisure,1,London Heathrow to Mumbai in a Boeing 787-8 in...,"[London, Heathrow, to, Mumbai, in, a, Boeing, ..."
3,"✅ Trip Verified | Keflavík, Iceland to London ...",2023-07-31,Iceland,Business Class,yes,5,London to Luanda,Couple Leisure,1,"Keflavík, Iceland to London Heathrow on an A32...","[Keflavík,, Iceland, to, London, Heathrow, on,..."
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,Economy Class,no,5,Denver to Heathrow,Family Leisure,1,Terrible Experience with British Airways. I bo...,"[Terrible, Experience, with, British, Airways...."


In [7]:
# Some routes have airport codes while others don't
# List of airports codes to standardize the information in departure and arrival locations
airport_codes = {
    "London": "LHR",
    "Heathrow": "LHR",
    "Las Vegas": "LAS",
    "Luanda": "LAD",
    "Denver": "DEN",
    "Rome": "FCO",
    "Gatwick": "LGW",
    "Venice": "VCE",
    "Tampa": "TPA",
    "Sydney": "SYD",
    "Bangkok": "BKK",
    "Cape Town": "CPT",
    "Bangalore": "BLR"
}

df = df.apply(lambda x: x.replace(airport_codes, regex=True))
df.head()

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller,trip_verified,reviews_verif_drop,reviews_bag_of_words
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,Business Class,yes,5,LHR to LAS,Family Leisure,1,I had the most fantastic BA Flight today. The ...,"[I, had, the, most, fantastic, BA, Flight, tod..."
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,Economy Class,no,3,FCO to LHR,Solo Leisure,1,Couldn’t book in online. Arrived at check in t...,"[Couldn’t, book, in, online., Arrived, at, che..."
2,✅ Trip Verified | LHR LHR to Mumbai in a Boein...,2023-07-31,Iceland,Business Class,yes,3,LGW to VCE,Solo Leisure,1,LHR LHR to Mumbai in a Boeing 787-8 in Busines...,"[London, Heathrow, to, Mumbai, in, a, Boeing, ..."
3,"✅ Trip Verified | Keflavík, Iceland to LHR LHR...",2023-07-31,Iceland,Business Class,yes,5,LHR to LAD,Couple Leisure,1,"Keflavík, Iceland to LHR LHR on an A320 in Bus...","[Keflavík,, Iceland, to, London, Heathrow, on,..."
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,Economy Class,no,5,DEN to LHR,Family Leisure,1,Terrible Experience with British Airways. I bo...,"[Terrible, Experience, with, British, Airways...."


In [8]:

# In the airline industry, seat pricing implies a given order. 
# So, we will encode accordingly
ordinal_encoder = ce.OrdinalEncoder(cols=['seat_type', 'stars'],return_df=True,
                       mapping=[
                           {'col':'seat_type',
                            'mapping':{'Economy Class':0,'Premium Economy':1,
                                            'Business Class':2,'First Class':3}},
                           {'col':'stars',
                            'mapping':{1:0, 3:1, 5:2,7:3, 9:4}}])


# 'recommended' and 'trip_verified' are binary categorical variables so, will be encoded as such
binary_encoder= ce.BinaryEncoder(cols='trip_verified',return_df=True)

## 'type_of_traveller' and 'route' are both nominal categorical variables
# With 4 and 10 categories respectively
#So we will one-hot-encoding
one_hot_encoder = ce.OneHotEncoder(cols=["type_of_traveller", "route"], use_cat_names=True,
                                   return_df=True)

# The 'country' column is categorical feature, but with 67 distinct entities
# To avoid increasing the dimensionality in the data set, we
# used frequency encoding. 
freq_encoder = ce.CountEncoder(cols=['country'], normalize=True, return_df=True)

In [9]:
def encoding_cat_vars(data, encoders):
    """ Encoding categorical variables

    Inputs
    -------
        data: dataframe
        encoders: list of dictionaries with the encoders

    Returns transformed dataframe
    """

    # Looping through all encoders for an in-place encoding
    for enc in encoders:
        # Define the key-value pair in context
        enc_name = list(enc.keys())[0]
        enc_vars = list(enc.values())[0]

        # Special Case for frequency encoding
        if enc_name == 'freq_encoder':
            # New columns from list of columns to encode 
            # with a frequency encoder
            col = enc_vars[0]
            func = enc_vars[1]
            data[col+'_encoded'] = func.fit_transform(data[col])

        # For all other cases, apply 'fit_transform' inplace
        else:
            func = enc_vars
            data = func.fit_transform(data)

    # New dataframe after encoding
    return data


# Applying all changes using the defined function above
df_encoded = encoding_cat_vars(data = df, 
                               encoders = [{'ordinal_encoder': ordinal_encoder},
                                           {'binary_encoder': binary_encoder},
                                           {'one_hot_encoder': one_hot_encoder},
                                           {'freq_encoder': ['country', freq_encoder]}
                                            ]
                               )

df_encoded.head()

,reviews,date,country,seat_type,recommended,stars,route_LHR to LAS,route_FCO to LHR,route_LGW to VCE,route_LHR to LAD,...,route_BLR to LHR,type_of_traveller_Family Leisure,type_of_traveller_Solo Leisure,type_of_traveller_Couple Leisure,type_of_traveller_Business,trip_verified_0,trip_verified_1,reviews_verif_drop,reviews_bag_of_words,country_encoded
0,✅ Trip Verified | I had the most fantastic BA...,2023-08-01,Hong Kong,2,yes,2,1,0,0,0,...,0,1,0,0,0,0,1,I had the most fantastic BA Flight today. The ...,"[I, had, the, most, fantastic, BA, Flight, tod...",0.0052
1,✅ Trip Verified | Couldn’t book in online. Ar...,2023-07-31,United Kingdom,0,no,1,0,1,0,0,...,0,0,1,0,0,0,1,Couldn’t book in online. Arrived at check in t...,"[Couldn’t, book, in, online., Arrived, at, che...",0.6208
2,✅ Trip Verified | LHR LHR to Mumbai in a Boein...,2023-07-31,Iceland,2,yes,1,0,0,1,0,...,0,0,1,0,0,0,1,LHR LHR to Mumbai in a Boeing 787-8 in Busines...,"[London, Heathrow, to, Mumbai, in, a, Boeing, ...",0.0012
3,"✅ Trip Verified | Keflavík, Iceland to LHR LHR...",2023-07-31,Iceland,2,yes,2,0,0,0,1,...,0,0,0,1,0,0,1,"Keflavík, Iceland to LHR LHR on an A320 in Bus...","[Keflavík,, Iceland, to, London, Heathrow, on,...",0.0012
4,✅ Trip Verified | Terrible Experience with Bri...,2023-07-29,Canada,0,no,2,0,0,0,0,...,0,1,0,0,0,0,1,Terrible Experience with British Airways. I bo...,"[Terrible, Experience, with, British, Airways....",0.0320


# Final Dataset

In [10]:
# After all pre-processing steps,
# Save onto a new csv file
#df_encoded.to_csv("BA_Flights_Reviews_processed.csv", header=True)

# Machine Learning

In [11]:
# Use recommended as target variable
# Use label encoding

# NLP

In [12]:
# Strategy 2: Perform Sentiment Analysis to create sentiment_label (positive or negative)
# Create wordcloud

# Business Analysis & Recommendations